##Install & import the necessary librairy : to be launch each time

In [ ]:
!pip install fuzzywuzzy
!pip install pyspellchecker
!apt-get install -y enchant
!pip install nltk pyenchant
!pip install enchant
!pip install autocorrect
!pip install langdetect
!pip install pyspellchecker
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package enchant
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 965.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=981463d46e32ce7e689d896edf7ff481b7ae3d4afe21f3668ea795a8946ef7b1
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=095cadb6d357cf6242a3d9890e8d3c11

In [ ]:
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # Import the required libraries
from bs4 import BeautifulSoup  # For parsing HTML
import requests  # For making HTTP requests
import pandas as pd  # For data manipulation and analysis
import time
import requests
import re
import sqlite3
from fuzzywuzzy import fuzz
from autocorrect import Speller
from langdetect import detect
import nltk
import langdetect

import os, time, math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import pprint  # for pretty printing
pp = pprint.PrettyPrinter()

##Data pre-cleaning process : rapper name variation : no need to launch again

In [ ]:
# code to see the rapper name variation in lyrics

# Initialize a list to store the original names, their variations found in the lyrics, and the similarity levels
name_variations_with_similarity = []

# Function to find similar names within the lyrics
def find_similar_names_in_lyrics(df):
    for index, row in df.iterrows():
        url = row['url']
        challenger_name = row['challenger_name']
        defender_name = row['defender_name']
        challenger_lyric = row['challenger_lyrics']
        defender_lyric = row['defender_lyrics']

        # Check for similar names within the challenger lyrics
        challenger_words = challenger_lyric.split()
        for word in challenger_words:
            similarity_level = fuzz.ratio(challenger_name.lower(), word.lower())
            if 80 <= similarity_level <= 99:
                name_variations_with_similarity.append((url, 'Challenger', challenger_name, word, similarity_level))

        # Check for similar names within the defender lyrics
        defender_words = defender_lyric.split()
        for word in defender_words:
            similarity_level = fuzz.ratio(defender_name.lower(), word.lower())
            if 80 <= similarity_level <= 99:
                name_variations_with_similarity.append((url, 'Defender', defender_name, word, similarity_level))

    # Convert the list to a DataFrame
    df_similar_names = pd.DataFrame(name_variations_with_similarity, columns=['URL', 'Role', 'Original_Name', 'Variation', 'Similarity_Level'])

    return df_similar_names

# Sample code to run the function (assuming 'sample_df_for_names' is your DataFrame)
df_similar_names = find_similar_names_in_lyrics(df_battles)

# Sample code to save the DataFrame to a Tableau file
df_similar_names.to_csv('/content/drive/MyDrive/Colab Notebooks/tableau_file.csv', index=False)


## Cleaning functions : to launch each time

In [ ]:
# Step -1 : Convert data into dataframe
# Function to convert the entire 'battles' table in the SQLite database to a Pandas DataFrame
def import_and_convert_to_dataframe(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    # Fetch all data from the table into a Pandas DataFrame
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql(query, conn)
    # Close the database connection
    conn.close()
    return df

# Step 0 : replace rapper name variation with the original name
def replace_rapper_name_variations_in_lyric(lyric, role, cleaned_df, url):
    # Extract the relevant rows from cleaned_df based on the URL and role
    relevant_rows = cleaned_df[(cleaned_df['URL'] == url) & (cleaned_df['Role'].str.lower() == role.lower())]

    # Replace variations with the original names in the lyric
    for _, row in relevant_rows.iterrows():
        original_name = row['Original_Name']
        variation = row['Variation']
        lyric = lyric.replace(variation, original_name)

    return lyric

import re

def step_replace_self_references(lyric, self_name):
    # Compile the regex pattern for an exact match.
    # The \b ensures we match full words only. re.IGNORECASE makes the match case-insensitive.
    pattern = re.compile(rf'\b{re.escape(self_name)}\b', re.IGNORECASE)

    # Replace the self_name with 'xxxxmyselfxxxx' wherever it appears as a full word in the lyric.
    modified_lyric = pattern.sub('xxxxmyselfxxxx', lyric)

    return modified_lyric

def step_replace_opponent_references(lyric, opponent_name):
    # Compile the regex pattern for an exact match.
    # The \b ensures we match full words only. re.IGNORECASE makes the match case-insensitive.
    pattern = re.compile(rf'\b{re.escape(opponent_name)}\b', re.IGNORECASE)

    # Replace the opponent_name with 'xxxxoponentxxxx' wherever it appears as a full word in the lyric.
    modified_lyric = pattern.sub('xxxxoponentxxxx', lyric)

    return modified_lyric


# Step : Convert to lowercase
def step_lowercase(lyric):
    return lyric.lower()

# Step : remove any trailing special characters, including "_", ".", and "-", at the end of lines, while keeping "!", "?", and ")".
def step_remove_trailing_special_chars(lyric):
    # Remove "#" symbols, even if they are attached to words
    lyric = re.sub(r'#', '', lyric)

    # Remove special characters like "_", ".", "-", and "," from the end of each line
    lyric = re.sub(r'[_\.,-]$', '', lyric, flags=re.MULTILINE)

    # Remove any other unwanted special characters at the end of lines, while keeping "!", "?", and ")"
    lyric = re.sub(r'[^!?)\w\s]$', '', lyric, flags=re.MULTILINE)

    return lyric

# Step 5: Replace ellipsis ("...", "..", "....", etc.) with a space
def step_replace_ellipsis(lyric):
    return re.sub(r'\.{2,}', ' ', lyric)

# Step 6: Remove special characters including emojis
def step_remove_special_chars(lyric):
    return re.sub(r'[@#$%^*+=|\\<>\/]', '', lyric)

# Step 7: Remove URLs
def step_remove_urls(lyric):
    return re.sub(r'http\S+|www\S+|https\S+', '', lyric, flags=re.MULTILINE)

# Step 8: Remove lines like "//Verse1" and "Verse X"
def step_remove_verse_labels(lyric):
    lyric = re.sub(r'//.*', '', lyric)
    return re.sub(r'\nverse \d+', '', lyric, flags=re.IGNORECASE)

# Step 9: Remove lines with only one word
def step_remove_one_word_lines(lyric):
    lines = lyric.strip().split('\n')
    cleaned_lines = [line for line in lines if len(line.split()) > 1]
    return '\n'.join(cleaned_lines)

# Step 10: Remove lyrics with fewer than 4 lines (considered as spam)
def step_remove_short_lyrics(lyric):
    lines = lyric.strip().split('\n')
    if len(lines) < 4:
        return None  # Returning None
    return '\n'.join(lines)

# Step 11: Remove consecutive duplicate lines, keeping just one
def step_remove_consecutive_duplicate_lines(lyric):
    lines = lyric.strip().split('\n')
    unique_lines = []
    prev_line = None

    for line in lines:
        if line != prev_line:
            unique_lines.append(line)
        prev_line = line

    return '\n'.join(unique_lines)

# Step 12: Remove consecutive spaces while preserving line breaks
def step_remove_consecutive_spaces(lyric):
    if lyric is not None:
        lines = lyric.split('\n')
        cleaned_lines = []

        for line in lines:
            cleaned_line = ' '.join(line.split())
            cleaned_lines.append(cleaned_line)

        return '\n'.join(cleaned_lines)
    else:
        return None  # Return None if the lyric is None

# Step 13: Tokenization
def step_tokenize(lyric):
    lines = lyric.strip().split('\n')
    tokenized_lines = [' '.join(word_tokenize(line)) for line in lines]
    return '\n'.join(tokenized_lines)

# Step 14: Spelling Correction
def step_spell_correction(lyric):
    special_words = {'xxxxoponentxxxx', 'xxxxmyselfxxxx'}

    lines = lyric.strip().split('\n')
    corrected_lines = []

    for line in lines:
        words = line.split()
        corrected_words = []

        for word in words:
            if word in special_words:
                corrected_words.append(word)
            else:
                corrected_word = spell.correction(word)
                if corrected_word is not None:
                    corrected_words.append(corrected_word)
                else:
                    corrected_words.append(word)

        corrected_line = ' '.join(corrected_words)
        corrected_lines.append(corrected_line)

    return '\n'.join(corrected_lines)



# Step : Lemmatization with spaCy
import spacy
nlp = spacy.load("en_core_web_sm")

def step_lemmatize(lyric):
    special_words = {'xxxxoponentxxxx', 'xxxxmyselfxxxx'}

    lines = lyric.strip().split('\n')
    lemmatized_lines = []

    for line in lines:
        words = line.split()
        #print(f"Debug: Before Lemmatization: {words}")  # Debug statement

        # Process each line with spaCy
        doc = nlp(line)

        # Lemmatize each token
        lemmatized_words = [token.lemma_ if token.text not in special_words else token.text for token in doc]

        #print(f"Debug: After Lemmatization: {lemmatized_words}")  # Debug statement

        lemmatized_line = ' '.join(lemmatized_words)
        lemmatized_lines.append(lemmatized_line)

    return '\n'.join(lemmatized_lines)

#Step : abbreviation
def expand_abbreviations(lyric):
    abbreviations = {
        " u ": " you ",
        " ur ": " your ",
        " cuz ": " because ",
        " cos ": " because ",
        " yr ": " your ",
        " y'all ": " you all ",
        " ain't ": " am not ",
        " gonna ": " going to ",
        " wanna ": " want to ",
        " gotta ": " got to ",
        " til ": " until ",
        " b4 ": " before ",
        " bout ": " about ",
        " luv ": " love ",
        " thx ": " thanks ",
        " pls ": " please ",
        " plz ": " please ",
        " fam ": " family ",
        " idk ": " I do not know ",
        " imo ": " in my opinion ",
        " imho ": " in my humble opinion ",
        " brb ": " be right back ",
        " gtg ": " got to go ",
        " btw ": " by the way ",
        " omg ": " oh my god ",
        " smh ": " shaking my head ",
        " tbh ": " to be honest ",
        " idc ": " I do not care ",
        " wyd ": " what you doing ",
        " hmu ": " hit me up ",
        " dm ": " direct message ",
        " yo ": " hey ",
        " bro ": " brother ",
        " sis ": " sister ",
        " r ": " are ",
        " n ": " and ",
        "'m ": " am ",
        "'ll ": " will ",
        " ill " : " I will ",
        "'ve ": " have ",
        "'re ": " are ",
        "n't ": " not ",
        "'s ": " is ",  # Note: this is ambiguous; could mean "has" or "is"
        " i' ": " I ",
        " i'am ": " I am ",
        " i " : " I ",
        " im ": " I am ",
        " im' " : " I am ",
        " wi ": " we ",
        " don't ": " do not ",
        " dont ": " do not ",
        " doesn't ": " does not ",
        " doesnt ": " does not ",
        " won't":" will not ",
        " make'em ": " make them",
        " ta ": " to ",
        " em ": " them ",
        " ca not " : " cannot ",
        " wont" : " will not "

    }

    for abbr, full in abbreviations.items():
        lyric = lyric.lower().replace(abbr, full)

    return lyric


# Step : remove non standart word
def step_remove_non_standard_words(lyric, standard_dict, custom_dict):
    special_words = {'xxxxoponentxxxx', 'xxxxmyselfxxxx'}
    lines = lyric.strip().split('\n')
    cleaned_lines = []
    removed_words = []  # List to keep track of removed words

    for line in lines:
        words = line.split()
        cleaned_words = []

        for word in words:
            if word in standard_dict or word in custom_dict or word in special_words:
                cleaned_words.append(word)
            else:
                removed_words.append(word)

        cleaned_line = ' '.join(cleaned_words)
        cleaned_lines.append(cleaned_line)

    return '\n'.join(cleaned_lines), removed_words

# Step 18: Remove Stop Words
def step_remove_stop_words(lyric):
    stop_words = set(stopwords.words('english'))
    lines = lyric.strip().split('\n')
    filtered_lines = []
    for line in lines:
        words = line.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        filtered_line = ' '.join(filtered_words)
        filtered_lines.append(filtered_line)
    return '\n'.join(filtered_lines)

# Step 19: Handle Contractions
def step_handle_contractions(lyric):
    contractions_dict = {"I'll": "I will", "they're": "they are"}  # Add more
    lines = lyric.strip().split('\n')
    expanded_lines = []
    for line in lines:
        for contraction, expansion in contractions_dict.items():
            line = line.replace(contraction, expansion)
        expanded_lines.append(line)
    return '\n'.join(expanded_lines)

# Step 20: Remove Numbers
def step_remove_numbers(lyric):
    return re.sub(r'\b\d+\b', '', lyric)

# Step 21: Remove Short Words
def step_remove_short_words(lyric):
    lines = lyric.strip().split('\n')
    filtered_lines = []
    for line in lines:
        words = line.split()
        filtered_words = [word for word in words if len(word) > 2]
        filtered_line = ' '.join(filtered_words)
        filtered_lines.append(filtered_line)
    return '\n'.join(filtered_lines)

# Step 22 : Remove Punctuation
def step_remove_punctuation(lyric):
    return re.sub(r'[^\w\s]', '', lyric)

# Step 23 : Language Detection
def step_language_detection(lyric):
    try:
        if langdetect.detect(lyric) == 'en':
            return lyric
    except langdetect.lang_detect_exception.LangDetectException:
        pass
    return None  # Return None if the lyric is not English or if there was an error


# Step 24: Convert British English to American English
def step_british_to_american(lyric, british_to_american_path):
    df = pd.read_csv(british_to_american_path)
    british_to_american_dict = dict(zip(df['British'], df['American']))
    for british, american in british_to_american_dict.items():
        pattern = re.compile(rf'\b{british}\b', re.IGNORECASE)
        lyric = pattern.sub(american, lyric)
    return lyric

# Final cleaning function using individual steps (Optimized Order)
def clean_lyric_optimized(lyric, self_name, opponent_name, url, cleaned_df, standard_dict, custom_dict, british_to_american_path):
    # Check if the lyric has fewer than 3 words
    if len(lyric.split()) < 3:
        return None, []

    # Step 1: Language Detection
    lyric = step_language_detection(lyric)
    if lyric is None:
        return None, []

    # Step 2: Replace Rapper Name Variations
    lyric = replace_rapper_name_variations_in_lyric(lyric, 'Challenger', cleaned_df, url)
    # Step 3: Replace Self and Opponent References
    lyric = step_replace_self_references(lyric, self_name)
    lyric = step_replace_opponent_references(lyric, opponent_name)
    # Step 4: Lowercase Conversion
    lyric = step_lowercase(lyric)
    # Step 5: Handle Contractions
    lyric = step_handle_contractions(lyric)
    # Step 6: Remove URLs, Special Characters, Emojis, Trailing Chars, Ellipsis, Verse Labels, One-Word Lines
    lyric = step_remove_urls(lyric)
    lyric = step_remove_special_chars(lyric)
    lyric = step_remove_trailing_special_chars(lyric)
    lyric = step_replace_ellipsis(lyric)
    lyric = step_remove_verse_labels(lyric)
    lyric = step_remove_one_word_lines(lyric)
    # Step 7: Remove Consecutive Duplicate Lines and Spaces
    lyric = step_remove_consecutive_duplicate_lines(lyric)
    lyric = step_remove_consecutive_spaces(lyric)
    lyric = step_remove_numbers(lyric)
    # Step 8: Spelling Correction
    lyric = step_spell_correction(lyric)
    lyric = expand_abbreviations(lyric)
    # Step 9: Lemmatization
    #lyric = step_lemmatize(lyric)

    # Step 10: Tokenization
    #lyric = step_tokenize(lyric)

    # Step 11: Remove Non-Standard Words, Stop Words, Numbers, Short Words, Punctuation
    lyric, removed_words = step_remove_non_standard_words(lyric, standard_dict, custom_dict)

    #lyric = step_remove_stop_words(lyric)
    #lyric = step_remove_short_words(lyric)
    lyric = step_remove_punctuation(lyric)

    # Step 12: Convert British to American English
    lyric = step_british_to_american(lyric, british_to_american_path)

    # Step 13: Remove Short Lyrics
    #lyric = step_remove_short_lyrics(lyric)

    return lyric, removed_words  # return both the cleaned lyric and the list of removed words


Import dictionary : to launch each time

In [ ]:
# Import the 'battles' table and convert it to a DataFrame
db_path = '/content/drive/MyDrive/Colab Notebooks/data.db'
table_name = 'battles'
df_battles = import_and_convert_to_dataframe(db_path, table_name)

# Import bristish to american mapping
british_to_american_path = '/content/drive/MyDrive/Colab Notebooks/British_to_American_Mappings.csv'

# Import the cleaned DataFrame rapper name variation (cleaned manually)
cleaned_csv_path = "/content/drive/MyDrive/Colab Notebooks/Rapper name variation cleaned.csv"
df_cleaned_filtered = pd.read_csv(cleaned_csv_path)

# Import Urban Rap dictionnay
custom_dict_path = '/content/drive/MyDrive/Colab Notebooks/urbandict-word-defs.csv'
custom_dict = pd.read_csv(custom_dict_path, error_bad_lines=False)['word'].str.lower().tolist()
custom_dict = set(custom_dict)  # Converting list to set for faster lookup

# nltk_words is the nltk English dictionary
nltk.download('words')
from nltk.corpus import words
standard_dict = set(words.words())


## Create word removed and cleaning lyric : to launch when needed time

In [ ]:
# Import necessary modules
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from spellchecker import SpellChecker

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Initialize SpellChecker
spell = SpellChecker()

# Sample 100 rows from the DataFrame
sample_df = df_battles.sample(n=100)

# Initialize an empty list for storing removed words
removed_words = []

# Open text files for storing cleaned lyrics and removed words
with open("/content/drive/MyDrive/Colab Notebooks/original_and_cleaned_lyrics.txt", "w") as f, \
     open("/content/drive/MyDrive/Colab Notebooks/removed_words.txt", "w") as removed_f, \
     open("/content/drive/MyDrive/Colab Notebooks/corpus_cleaned_lyrics.txt", "w") as corpus_f:

    for index, row in sample_df.iterrows():
        try:
            # Extract data from the DataFrame row
            url = row['url']
            challenger_name = row['challenger_name']
            defender_name = row['defender_name']
            original_challenger_lyric = row['challenger_lyrics']
            original_defender_lyric = row['defender_lyrics']

            # Clean the lyrics for the challenger
            cleaned_challenger_lyric, removed_words_challenger = clean_lyric_optimized(
                original_challenger_lyric, challenger_name, defender_name, url, df_cleaned_filtered, standard_dict, custom_dict, british_to_american_path)

            # Clean the lyrics for the defender
            cleaned_defender_lyric, removed_words_defender = clean_lyric_optimized(
                original_defender_lyric, defender_name, challenger_name, url, df_cleaned_filtered, standard_dict, custom_dict, british_to_american_path)

            # Write cleaned lyrics to the file if they are not None
            if cleaned_challenger_lyric and cleaned_defender_lyric:
                f.write(f"Challenger: {challenger_name}\n")
                for orig_line, cleaned_line in zip(original_challenger_lyric.split('\n'), cleaned_challenger_lyric.split('\n')):
                    f.write(f"Original: {orig_line}\n")
                    f.write(f"Cleaned: {cleaned_line}\n")

                f.write(f"Defender: {defender_name}\n")
                for orig_line, cleaned_line in zip(original_defender_lyric.split('\n'), cleaned_defender_lyric.split('\n')):
                    f.write(f"Original: {orig_line}\n")
                    f.write(f"Cleaned: {cleaned_line}\n")

                f.write("=" * 50 + "\n")

            # Write cleaned lyrics to the corpus file
            if cleaned_challenger_lyric and cleaned_defender_lyric:
                corpus_f.write(cleaned_challenger_lyric + "\n")
                corpus_f.write(cleaned_defender_lyric + "\n")

            # Write removed words to a separate file
            removed_f.write(f"Challenger: {challenger_name}\n")
            removed_f.write("\n".join(removed_words_challenger) + "\n")
            removed_f.write(f"Defender: {defender_name}\n")
            removed_f.write("\n".join(removed_words_defender) + "\n")
            removed_f.write("=" * 50 + "\n")

        except Exception as e:
            # Log exceptions
            removed_f.write(f"Error at index {index}: {e}\n")
            removed_f.write("=" * 50 + "\n")


## create corpus only

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
from spellchecker import SpellChecker
import pandas as pd

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Initialize SpellChecker
spell = SpellChecker()

# Initialize new columns for cleaned lyrics
df_battles['cleaned_challenger_lyric'] = ''
df_battles['cleaned_defender_lyric'] = ''

# Number of samples in each batch
batch_size = 5000

# Calculate total batches
total_batches = math.ceil(len(df_battles) / batch_size)

# Loop through all batches
for batch in range(3, total_batches):  # Adjust range as needed

    # Calculate the start and end indices for the current batch
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, len(df_battles))

    # Create a new file for each batch
    file_name = f"/content/drive/MyDrive/Colab Notebooks/corpus_cleaned_lyrics_batch_{batch}.txt"
    with open(file_name, "w") as corpus_f:

        # Loop through the rows in the current batch
        for idx, (index, row) in enumerate(df_battles.iloc[start_idx:end_idx].iterrows()):

            try:
                # Extract data from the DataFrame row
                url = row['url']
                challenger_name = row['challenger_name']
                defender_name = row['defender_name']
                original_challenger_lyric = row['challenger_lyrics']
                original_defender_lyric = row['defender_lyrics']

                # Clean the lyrics for the challenger
                cleaned_challenger_lyric, _ = clean_lyric_optimized(
                    original_challenger_lyric, challenger_name, defender_name, url, df_cleaned_filtered, standard_dict, custom_dict, british_to_american_path)
                cleaned_challenger_lyric = cleaned_challenger_lyric or " "

                # Clean the lyrics for the defender
                cleaned_defender_lyric, _ = clean_lyric_optimized(
                    original_defender_lyric, defender_name, challenger_name, url, df_cleaned_filtered, standard_dict, custom_dict, british_to_american_path)
                cleaned_defender_lyric = cleaned_defender_lyric or " "

                # Update the DataFrame
                df_battles.at[index, 'cleaned_challenger_lyric'] = cleaned_challenger_lyric
                df_battles.at[index, 'cleaned_defender_lyric'] = cleaned_defender_lyric

                # Write cleaned lyrics to the corpus file
                corpus_f.write(cleaned_challenger_lyric + "\n")
                corpus_f.write(cleaned_defender_lyric + "\n")

                # Print progress
                if (idx + 1) % 5000 == 0:
                    print(f"Processed {start_idx + idx + 1} rows.")

            except Exception as e:
                print(f"Error at index {start_idx + idx}: {e}")

    # Save the updated DataFrame to a new CSV file, unique for each batch
    df_battles.iloc[start_idx:end_idx].to_csv(f"/content/drive/MyDrive/Colab Notebooks/battles_cleaned_batch_{batch}.csv", index=False)

    # Optional: Print completion of batch
    print(f"Batch {batch} completed and saved.")







